# LoL Win-Loss Prediction Model

**Name(s)**: Minchan Kim, David Moon

**Website Link**: https://m1nce.github.io/wrmodel/

## Code

In [112]:
import pandas as pd
import numpy as np
import os
import pathlib
from pathlib import Path
import plotly.express as px
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
pd.options.plotting.backend = 'plotly'

In [113]:
data_path = Path('data') / '2022_LoL_esports_match_data_from_OraclesElixir.csv'
df = pd.read_csv(data_path,low_memory=False)
df.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0
1,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,119.0,-475.0,153.0,1.0,0.0,3.0,0.0,3.0,3.0,2.0
3,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,149.0,-793.0,-1343.0,-34.0,2.0,1.0,2.0,3.0,3.0,0.0
4,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,21.0,443.0,-497.0,7.0,1.0,2.0,2.0,0.0,6.0,2.0


### Framing the Problem

For our analysis, we decided to predict whether or not a team will win or lose. We decided to use a binary classification because a game only has two outcomes, a win or a loss. Because this is the way games are set up, we will consider a win to be the number "1" and a loss to be the number "0".

In [114]:
is_lck = 'LCK'
default = df.loc[:, ['gameid', 'league','position','side', 'result', 'golddiffat10','firstherald','firstdragon','firstblood','xpdiffat10','csdiffat10']]
default = default[default['position'] == 'team']
#gets rid of games that have no result
groupbyresult = default.groupby(['gameid'])[['result']].sum()
remove_games = groupbyresult[groupbyresult['result'] != 6].index
default = default[default.isin(remove_games).any(axis=1)]
default = default[default['league'] == is_lck]
default

,gameid,league,position,side,result,golddiffat10,firstherald,firstdragon,firstblood,xpdiffat10,csdiffat10
1450,ESPORTSTMNT01_2700815,LCK,team,Blue,0,281.0,1.0,0.0,1.0,404.0,6.0
1451,ESPORTSTMNT01_2700815,LCK,team,Red,1,-281.0,0.0,1.0,0.0,-404.0,-6.0
1462,ESPORTSTMNT01_2690695,LCK,team,Blue,0,-1200.0,1.0,0.0,0.0,-1277.0,-15.0
1463,ESPORTSTMNT01_2690695,LCK,team,Red,1,1200.0,0.0,1.0,1.0,1277.0,15.0
1486,ESPORTSTMNT01_2690705,LCK,team,Blue,1,152.0,1.0,1.0,1.0,-948.0,-13.0
...,...,...,...,...,...,...,...,...,...,...,...
129503,ESPORTSTMNT04_2451974,LCK,team,Red,1,578.0,0.0,1.0,0.0,757.0,29.0
129538,ESPORTSTMNT04_2472110,LCK,team,Blue,0,-2261.0,0.0,0.0,0.0,-1791.0,6.0
129539,ESPORTSTMNT04_2472110,LCK,team,Red,1,2261.0,1.0,1.0,1.0,1791.0,-6.0
129562,ESPORTSTMNT04_2472111,LCK,team,Blue,1,308.0,0.0,1.0,0.0,956.0,20.0


### Baseline Model

For our baseline model, we wanted to predict a team's outcome (win or loss) depending on a couple of features. We used the `side`, `golddiffat10`, `firstherald`, `firstdragon`, `firstblood`, `xpdiffat10`, and `csdiffat10` columns in order to predict the outcome.

In [202]:
X = default.drop(columns=['gameid','league','position','result'])
y = default['result']
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [203]:
preproc = ColumnTransformer(
    transformers=[
        ('categorical_col', OneHotEncoder(), ['side'])
    ],
    remainder='passthrough'
)

In [204]:
pl = Pipeline([
    ('preprocessor', preproc),
    ('clf', DecisionTreeClassifier(max_depth=3)) 
])

In [205]:
pl.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical_col',
                                                  OneHotEncoder(),
                                                  ['side'])])),
                ('clf', DecisionTreeClassifier(max_depth=3))])

In [206]:
pl.score(X_train, y_train)

0.6914285714285714

In [207]:
pl.score(X_test, y_test)

0.7094017094017094

### Final Model

In [ ]:
# TODO

### Fairness Analysis

In [ ]:
# TODO